### This script computes the shift model for the company.

The model is computed after analyzing raw data sources of employee's working hours. The feature extracted is a valuable context variable which allows for analyzing the performance of employee groups during different shift times. The model is computed using quantitative analysis.

##### Load libraries

In [1]:
from pathlib import Path
import pandas as pd

##### Read raw data sources

In [ ]:
emp = pd.read_csv(Path().joinpath('data', 'Data_1_RAW.csv'))
emp2 = pd.read_csv(Path().joinpath('data', 'Data_2_RAW.csv'))

##### Preparing the data

In [5]:
# Convert to datetime
emp['#'] = pd.to_datetime(emp['#'])
emp2['#'] = pd.to_datetime(emp2['#'])
emp['#'] = pd.to_datetime(emp['#'])
emp2['#'] = pd.to_datetime(emp2['#'])

# Round the hours
emp['#'] = emp['#'].dt.round('2H')
emp['#'] = emp['#'].dt.round('2H')

emp2['#'] = emp2['#'].dt.round('2H')
emp2['#'] = emp2['#'].dt.round('2H')

# Extract variables from dates
emp['#'] = emp['#'].dt.day_name()
emp['#'] = emp['#'].dt.day_name()
emp['#'] = emp['#'].dt.hour
emp['#'] = emp['#'].dt.hour
# and
emp2['#'] = emp2['#'].dt.day_name()
emp2['#'] = emp2['#'].dt.day_name()
emp2['#'] = emp2['#'].dt.hour
emp2['#'] = emp2['#'].dt.hour

##### Function that computes the shift model

In [6]:
def shift_model(data):
    # Shift Concept
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    col_names = ['Days', 'Start', 'End']
    d = []
    for i in range(len(days)):
        if i == days.index('Monday'):
            temp = (data['start_day'] == days[i]) & (data['end_day'] == days[i])
            temp = data.loc[temp]
            t = temp.groupby('start_hour')['end_hour'].value_counts().idxmax()
            d.append([days[i] + '-' + days[i], t[0], t[1]])

            temp = (data['start_day'] == days[i]) & (data['end_day'] == days[i + 1])
            temp = data.loc[temp]
            t = temp.groupby('start_hour')['end_hour'].value_counts().idxmax()
            d.append([days[i] + '-' + days[i + 1], t[0], t[1]])
        elif i == (len(days) - 1):
            temp = (data['start_day'] == days[i]) & (data['end_day'] == days[i])
            temp = data.loc[temp]
            t = temp.groupby('start_hour')['end_hour'].value_counts().nlargest(2).index
            d.append([days[i] + '-' + days[i], t[0][0], t[0][1]])
            d.append([days[i] + '-' + days[i], t[1][0], t[1][1]])

            temp = (data['start_day'] == days[i]) & (data['end_day'] == days[0])
            temp = data.loc[temp]
            t = temp.groupby('start_hour')['end_hour'].value_counts().idxmax()
            d.append([days[i] + '-' + days[0], t[0], t[1]])
        else:
            temp = (data['start_day'] == days[i]) & (data['end_day'] == days[i])
            temp = data.loc[temp]
            t = temp.groupby('start_hour')['end_hour'].value_counts().nlargest(2).index
            d.append([days[i] + '-' + days[i], t[0][0], t[0][1]])
            d.append([days[i] + '-' + days[i], t[1][0], t[1][1]])

            temp = (data['start_day'] == days[i]) & (data['end_day'] == days[i + 1])
            temp = data.loc[temp]
            t = temp.groupby('start_hour')['end_hour'].value_counts().idxmax()
            d.append([days[i] + '-' + days[i + 1], t[0], t[1]])

    shift_concept = pd.DataFrame(d, columns=col_names)
    return shift_concept

##### Shift model for maintenance employees

In [8]:
emp_shift_model = shift_model(emp)
emp_shift_model

,Days,Start,End
0,Monday-Monday,6,18
1,Monday-Tuesday,18,6
2,Tuesday-Tuesday,14,22
3,Tuesday-Tuesday,6,14
4,Tuesday-Wednesday,22,6
5,Wednesday-Wednesday,14,22
6,Wednesday-Wednesday,6,14
7,Wednesday-Thursday,22,6
8,Thursday-Thursday,6,14
9,Thursday-Thursday,14,22


##### Shift model for quality control employees

In [9]:
emp2_shift_model = shift_model(emp2)
emp2_shift_model

,Days,Start,End
0,Monday-Monday,6,18
1,Monday-Tuesday,18,6
2,Tuesday-Tuesday,14,22
3,Tuesday-Tuesday,6,14
4,Tuesday-Wednesday,22,6
5,Wednesday-Wednesday,6,14
6,Wednesday-Wednesday,14,22
7,Wednesday-Thursday,22,6
8,Thursday-Thursday,6,14
9,Thursday-Thursday,14,22


After analyzing these shift models, we see that 5 shift times are visible: early 8 and 12 hour shifts, late 8 hour shifts, night 8 and 12 hour shifts. They are labeled as (E8, E12, L8, N8 and N12) in the feature extraction step.